# DS321: Computational Statistics <br>

##   Lecture: Metropolis-Hastings Algorithm

University of Science and Technology of Southern Philippines <br>

## Student Name: <code>Romen Samuel Wabina</code>

Instructor: **Romen Samuel Wabina, MSc** <br>
MSc Data Science and AI | Asian Institute of Technology <br>
*ongoing* PhD Data Science (Healthcare and Clinical Informatics) 

MCMC algorithms are algorithms that samples from complex probability distributions. They are commonly used when it is difficult or impossible to directly sample from a target probability distribution. 

A Markov Chain is is a chain of discrete events where the probability of the next event is conditioned only upon the current event. In this system of discrete choices, there exists a transition matrix, which quantifies the probability of transitioning from any given state to any given state. A Monte Carlo method is really just a fancy name for a simulation/experiment that relies on usage of random numbers.

## Introduction: Metropolis-Hastings Algorithm

The Metropolis-Hastings algorithm is a Markov Chain Monte Carlo (MCMC) method used to generate samples from a probability distribution when direct sampling is difficult or infeasible. It is named after the scientists Nicholas Metropolis and W. Keith Hastings, who introduced the algorithm in the 1950s. The key idea of the algorithm is that it constructs a Markov chain in which the samples are generated sequentially, and the distribution of the samples converges to the desired target distribution over time. The acceptance probability ensures that the chain explores the sample space in a way that converges to the correct distribution.


Let's take a look at this Bayes' formula:
$$ P(\theta|x) = \frac{P(x|\theta) P(\theta)}{P(x)} $$

## Direct Sampling

Let's take a look at this Bayes' formula:
$$ P(\theta|x) = \frac{P(x|\theta) P(\theta)}{P(x)} $$

In [23]:
%matplotlib inline

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [24]:
import numpy as np

def monte_carlo_markov_chain(transition_matrix, initial_state, num_samples, num_steps):
    samples = []
    
    for _ in range(num_samples):
        current_state = initial_state
        states = [current_state]

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]

            # Direct Sampling
            next_state = np.random.choice(len(transition_probabilities), p = transition_probabilities)
            current_state = next_state
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break

        samples.append(states)
    return samples

# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]])

initial_state = 0   # Starting from Stage 3
num_samples = 10    # Number of Monte Carlo samples
num_steps = 10      # Number of steps to sample

# Perform Monte Carlo sampling with the Markov Chain
sampled_states = monte_carlo_markov_chain(transition_matrix, initial_state, num_samples, num_steps)

# Print the sampled states for the first sample
print("Sampled States (First Sample):", sampled_states[0])

Sampled States (First Sample): [0, 0, 2, 3, 4]


In [25]:
for idx, state in enumerate(sampled_states):
    print(f'Patient {idx} with CKD Progression: \t {set(state)}')

Patient 0 with CKD Progression: 	 {0, 2, 3, 4}
Patient 1 with CKD Progression: 	 {0, 1, 2}
Patient 2 with CKD Progression: 	 {0, 1, 2}
Patient 3 with CKD Progression: 	 {0, 1, 2, 3, 4}
Patient 4 with CKD Progression: 	 {0, 1, 2, 3}
Patient 5 with CKD Progression: 	 {0, 1, 2, 3}
Patient 6 with CKD Progression: 	 {0, 1, 2, 4}
Patient 7 with CKD Progression: 	 {0, 2, 4}
Patient 8 with CKD Progression: 	 {0, 1, 2, 4}
Patient 9 with CKD Progression: 	 {0, 1, 3, 4}


In the context of a Markov Chain model for chronic kidney disease progression, direct sampling is not inherently difficult because the Markov Chain allows for the direct generation of samples. The transition probabilities between different stages of the disease and other health states determine the probabilities of transitioning from one state to another. However, direct sampling have disadvantages:

1. Direct sampling can become challenging in high-dimensional spaces due to the "curse of dimensionality." Obtaining representative samples becomes computationally expensive or infeasible as the number of dimensions increases.
2. In some cases, the distribution of CKD stages and associated risk factors may be analytically or computationally intractable, making direct sampling challenging. If the distributions cannot be easily parameterized or sampled from, alternative methods like importance sampling or MCMC techniques may be more suitable.
3. Direct sampling can be computationally expensive, especially if the CKD model involves complex calculations, simulations, or computationally intensive algorithms. Sampling directly from a large dataset or running extensive simulations may not be feasible in real-time applications or when dealing with a massive amount of data.

## Importance Sampling

Importance sampling is a Markov Chain Monte Carlo method for evaluating properties of a particular distribution, while only having samples generated from a different distribution than the distribution of interest. Importance sampling is a variance reduction technique that can be used in the Monte Carlo method. The idea behind importance sampling is that certain values of the input random variables in a simulation have more impact on the parameter being estimated than others. If these "important" values are emphasized by sampling more frequently, then the estimator variance can be reduced. Hence, the basic methodology in importance sampling is to choose a distribution which "encourages" the important values.


With importance sampling, we try to reduce the variance of our Monte-Carlo integral estimation by choosing a better distribution from which to simulate our random variables. It involves multiplying the integrand by 1 (usually dressed up in a “tricky fashion”) to yield an expectation of a quantity that varies less than the original integrand over the region of integration. Concretely,

$$\mathbb{E}_{p(x)} \big[\ f(x) \big] = \int f(x)\ p(x)\ dx = \int f(x)\ p(x)\ \frac{q(x)}{q(x)}\ dx = \int \frac{p(x)}{q(x)}\cdot f(x)\ q(x)\ dx = \mathbb{E}_{q(x)}  \big[\ f(x)\cdot \frac{p(x)}{q(x)} \big]$$

Thus, the MC estimation of the expectation becomes:

$$\mathbb{E}_{q(x)}  \big[\ f(x)\cdot \frac{p(x)}{q(x)} \big] \approx \frac{1}{N} \sum_{n=1}^{N} w_n \cdot f(x_n)$$

where $w_n = \dfrac{p(x_n)}{q(x_n)}$


In importance sampling, the proposal distribution is a probability distribution used to generate samples during the sampling process. It serves as an approximation to the target distribution from which you want to estimate properties or calculate expectations. The proposal distribution should ideally have a similar shape or be closely related to the target distribution to ensure accurate importance sampling estimates. The primary purpose of the proposal distribution is to efficiently explore the space of possible samples and assign appropriate weights to each sample. The importance weights, which are the ratios of the target distribution's density to the proposal distribution's density, are used to adjust the contribution of each sample to the final estimate.

In [41]:
def importance_sampling_markov_chain(transition_matrix, proposal_matrix, initial_state, num_samples, num_steps):
    samples = []
    weights = []
    
    for _ in range(num_samples):
        current_state = initial_state
        states = [current_state]
        weight = 1.0

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]
            proposal_probabilities   = proposal_matrix[current_state]
            
            # Sample the next state using the proposal distribution
            next_state = np.random.choice(len(proposal_probabilities), p = proposal_probabilities)
            
            # Calculate the weight using the importance sampling ratio
            weight *= transition_probabilities[next_state] / proposal_probabilities[next_state]
            
            current_state = next_state
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break
        
        samples.append(states)
        weights.append(weight)
    
    # Normalize the weights
    weights /= np.sum(weights)
    
    return samples, weights

# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

# Proposal matrix representing the proposal distribution (can be different from the transition matrix)
# The proposal distribution is a probability distribution used to generate samples during the sampling 
# process. It serves as an approximation to the target distribution from which you want to estimate 
# properties or calculate expectations. The proposal distribution should ideally have a similar shape 
# or be closely related to the target distribution to ensure accurate importance sampling estimates.
proposal_matrix = np.array([
    [0.3, 0.3, 0.2, 0.1, 0.1],
    [0.1, 0.5, 0.2, 0.1, 0.1],
    [0.0, 0.1, 0.6, 0.2, 0.1],
    [0.0, 0.0, 0.1, 0.6, 0.3],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

initial_state = 0  # Starting from Stage 3
num_samples = 10   # Number of importance samples
num_steps = 10     # Number of steps to sample

# Perform importance sampling with the Markov Chain
sampled_states, importance_weights = importance_sampling_markov_chain(
    transition_matrix, proposal_matrix, initial_state, num_samples, num_steps)

In [45]:
for idx, state in enumerate(sampled_states):
    print(f'Patient {idx} with CKD Progression: \t {set(state)}')

Patient 0 with CKD Progression: 	 {0, 2, 3, 4}
Patient 1 with CKD Progression: 	 {0, 2, 4}
Patient 2 with CKD Progression: 	 {0, 1, 2, 4}
Patient 3 with CKD Progression: 	 {0, 1, 2, 3}
Patient 4 with CKD Progression: 	 {0, 2, 3, 4}
Patient 5 with CKD Progression: 	 {0, 1, 2, 4}
Patient 6 with CKD Progression: 	 {0, 2, 3, 4}
Patient 7 with CKD Progression: 	 {0, 1, 3, 4}
Patient 8 with CKD Progression: 	 {0, 3, 4}
Patient 9 with CKD Progression: 	 {0, 1, 3}


In [46]:
import numpy as np

def target_distribution(x):
    """Target distribution: Gaussian with mean 5 and standard deviation 2."""
    return np.exp(-(x - 5) ** 2 / 8) / np.sqrt(8 * np.pi)

def proposal_distribution():
    """Proposal distribution: Uniform distribution between 0 and 10."""
    return np.random.uniform(0, 10)

def importance_sampling(n_samples):
    """Implementation of Importance Sampling."""
    samples = np.zeros(n_samples)
    weights = np.zeros(n_samples)

    for i in range(n_samples):
        sample = proposal_distribution()
        weight = target_distribution(sample) / proposal_distribution()
        samples[i] = sample
        weights[i] = weight

    weights /= np.sum(weights)  # Normalize the weights
    estimated_mean = np.sum(samples * weights)

    return estimated_mean

def metropolis_hastings(n_samples, burn_in=1000):
    """Implementation of Metropolis-Hastings."""
    samples = np.zeros(n_samples + burn_in)
    accepted = 0
    current_sample = proposal_distribution()

    for i in range(n_samples + burn_in):
        proposal = proposal_distribution()
        acceptance_prob = min(1, target_distribution(proposal) / target_distribution(current_sample))

        if np.random.rand() < acceptance_prob:
            current_sample = proposal
            accepted += 1

        samples[i] = current_sample

    samples = samples[burn_in:]
    estimated_mean = np.mean(samples)

    return estimated_mean, accepted / (n_samples + burn_in)

# Testing the implementations
np.random.seed(42)
n_samples = 10000

# Importance Sampling
estimated_mean_importance = importance_sampling(n_samples)
print("Importance Sampling:")
print("Estimated Mean:", estimated_mean_importance)

# Metropolis-Hastings
estimated_mean_mh, acceptance_ratio = metropolis_hastings(n_samples)
print("\nMetropolis-Hastings:")
print("Estimated Mean:", estimated_mean_mh)
print("Acceptance Ratio:", acceptance_ratio)

Importance Sampling:
Estimated Mean: 4.815660868077247

Metropolis-Hastings:
Estimated Mean: 4.984573440505409
Acceptance Ratio: 0.614909090909091


In [47]:
import numpy as np

def importance_sampling_markov_chain(transition_matrix, pdf_func, initial_state, num_samples, num_steps):
    samples = []
    weights = []
    
    for _ in range(num_samples):
        current_state = initial_state
        states = [current_state]
        weight = 1.0

        for _ in range(num_steps):
            transition_probabilities = transition_matrix[current_state]
            
            # Generate a sample from the proposal distribution using the given PDF
            proposal_probabilities = pdf_func(current_state)
            next_state = np.random.choice(len(proposal_probabilities), p=proposal_probabilities)
            
            # Calculate the weight using the importance sampling ratio
            weight *= transition_probabilities[next_state] / proposal_probabilities[next_state]
            
            current_state = next_state
            states.append(current_state)

            # Stop sampling if a terminal state (e.g., Death) is reached
            if current_state == len(transition_matrix) - 1:
                break
        
        samples.append(states)
        weights.append(weight)
    
    # Normalize the weights
    weights /= np.sum(weights)
    
    return samples, weights

# Transition matrix representing the Markov Chain
transition_matrix = np.array([
    [0.5, 0.4, 0.1, 0.0, 0.0],
    [0.0, 0.6, 0.3, 0.1, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.8, 0.2],
    [0.0, 0.0, 0.0, 0.0, 1.0]
])

initial_state = 0  # Starting from Stage 3
num_samples = 1000  # Number of importance samples
num_steps = 10  # Number of steps to sample

# Define the proposal probability density function (PDF)
def proposal_pdf(state):
    # Example: Uniform distribution as the proposal PDF
    num_states = len(transition_matrix)
    probabilities = np.full(num_states, 1/num_states)
    return probabilities

# Perform importance sampling with the Markov Chain
sampled_states, importance_weights = importance_sampling_markov_chain(
    transition_matrix, proposal_pdf, initial_state, num_samples, num_steps
)

# Print the sampled states and their corresponding importance weights for the first sample
print("Sampled States (First Sample):", sampled_states[0])
print("Importance Weights (First Sample):", importance_weights[0])


Sampled States (First Sample): [0, 4]
Importance Weights (First Sample): 0.0
